In [1]:
# !pip3 install textgrid
import textgrid

In [2]:
import os

os.environ['AWS_ENDPOINT_URL'] = 'https://s3.us-west-000.backblazeb2.com'
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

In [3]:
from tqdm import tqdm
import boto3
import subprocess
from unidecode import unidecode
import IPython.display as ipd
from glob import glob
import soundfile as sf

os.makedirs('part3-same', exist_ok = True)
os.makedirs('part3-same-audio', exist_ok = True)
os.makedirs('part3-separate', exist_ok = True)
os.makedirs('part3-separate-audio', exist_ok = True)

In [59]:
!mkdir part3-same-audio-mp3
!mkdir part3-separate-audio-mp3
!rm part3-same-audio-mp3/*.mp3
!rm part3-separate-audio-mp3/*.mp3

mkdir: cannot create directory ‘part3-same-audio-mp3’: File exists
mkdir: cannot create directory ‘part3-separate-audio-mp3’: File exists
rm: cannot remove 'part3-separate-audio-mp3/*.mp3': No such file or directory


In [6]:
session = boto3.Session()

In [7]:
s3 = session.resource('s3')
bucket = s3.Bucket('imda-singapore')

In [8]:
for f in tqdm(bucket.objects.filter(Prefix='PART3/Scripts Same')):
    filename = os.path.join('part3-same', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

494it [00:00, 676.41it/s]


In [9]:
for f in tqdm(bucket.objects.filter(Prefix='PART3/Audio Same CloseMic')):
    filename = os.path.join('part3-same-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

492it [1:09:40,  8.50s/it]


In [10]:
for f in tqdm(bucket.objects.filter(Prefix='PART3/Scripts Separate')):
    filename = os.path.join('part3-separate', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

418it [00:00, 669.87it/s]


In [11]:
for f in tqdm(bucket.objects.filter(Prefix='PART3/Audio Separate StandingMic')):
    filename = os.path.join('part3-separate-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

431it [00:00, 1638.94it/s]


In [66]:
import re

def cleaning(s):
    return re.sub(r'[ ]+', ' ', s).strip()

replaces = [
    '<malay>',
    '</malay>',
    '_',
    '<UNK>',
    '(ppl)',
    '(ppo)',
    '(mm)',
    '(um)'
]

replaces_with = {
    '[lah]': 'lah',
    '[ah]': 'ah',
    '[sia]': 'sia',
    '[eh]': 'eh',
    '(uh)': 'uh',
    '[what]': 'what',
    '[oh]': 'oh',
    '(err)': 'err',
    '[lor]': 'lor',
    '[ha]': 'ha',
    '[meh]': 'meh',
    '[one]': 'one',
    "[a'ah]": "a'ah",
    '[hor]': 'hor',
    '[leh]': 'leh',
    '[mah]': 'mah',
    '[nah]': 'nah',
    '[tau]': 'tau',
    '[uh]': 'uh',
    '[wah]': 'wah'
}

maxlen = 15

In [13]:
scripts = sorted(glob('part3-same/*'))

all_results = {}
for s in tqdm(scripts):
    results = []
    id_file = os.path.split(s)[1].replace('.TextGrid', '')
    texts = []
    total_l = 0
    start = []
    end = []
    try:
        tg = textgrid.TextGrid.fromFile(s)
    except:
        continue
    for tg_ in tg[0]:
        if tg_.mark is None:
            continue
        if not len(tg_.mark):
            continue
        start.append(tg_.minTime)
        end.append(tg_.maxTime)
        total_l += (tg_.maxTime - tg_.minTime)
        if tg_.mark[0] != '<' and tg_.mark[-1] != '>':
            t = tg_.mark.split()
            filtered = []
            for no, t_ in enumerate(t):
                if t_[-1] == '~':
                    continue
                if t_[0] == '#' and t_[-1] == '#':
                    t_ = t_[1:-1].title()
                filtered.append(t_)
                t[no] = filtered

            texts.append(' '.join(filtered))
        if total_l >= maxlen:
            t = ', '.join(texts)
            if len(t) > 3:
                for r in replaces:
                    t = t.replace(r, '')
                for r, v in replaces_with.items():
                    t = t.replace(r, v)
                t = cleaning(t)
                results.append((t, [start[0], end[-1]]))
            texts = []
            total_l = 0
            start = []
            end = []
    
    if len(texts):
        t = ', '.join(texts)
        if len(t) > 3:
            for r in replaces:
                t = t.replace(r, '')
            for r, v in replaces_with.items():
                t = t.replace(r, v)
            t = cleaning(t)
            results.append((', '.join(texts), [start[0], end[-1]]))
    
    all_results[id_file] = results

In [25]:
keys = list(all_results)
keys = [(k, all_results[k]) for k in keys]

In [60]:
def loop(keys):
    results = []
    keys, _ = keys
    for key in tqdm(keys):
        key, rows = key
        audio_file = os.path.join('part3-same-audio', f'{key}.wav')
        if not os.path.exists(audio_file):
            continue
        y, sr = sf.read(audio_file)
        index = 0
        for row in rows:
            filename = os.path.join('part3-same-audio-mp3', f'{key}-{index}.mp3')
            if os.path.exists(filename):
                index += 1
                
                results.append({
                    'filename': filename,
                    'text': row[0]
                })
                
                continue
                
            y_ = y[int(sr * row[-1][0]): int(sr * row[-1][-1])]
            sf.write(filename, y_, sr)
            index += 1
            results.append({
                'filename': filename,
                'text': row[0]
            })
    return results

In [61]:
import mp

results = mp.multiprocessing(keys, loop, cores = 20)

100%|██████████| 18/18 [19:26<00:00, 64.83s/it]


In [62]:
len(results)

185448

In [63]:
results[-1]

{'filename': 'part3-same-audio-mp3/3273-2-380.mp3',
 'text': "GLD what's that, uh eh I think, uh maybe SGD is my favorite yes acronym , because it talks, UOB no lah actually because it talks SGD talks"}

In [64]:
ipd.Audio(results[-1]['filename'], rate = 16000)

In [65]:
import json

with open('imda-same-part3.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 185448/185448 [00:00<00:00, 296146.18it/s]


In [77]:
!du -hs part3-same-audio-mp3

17G	part3-same-audio-mp3


In [67]:
scripts = sorted(glob('part3-separate/*'))

all_results = {}
for s in tqdm(scripts):
    results = []
    id_file = os.path.split(s)[1].replace('.TextGrid', '')
    texts = []
    total_l = 0
    start = []
    end = []
    try:
        tg = textgrid.TextGrid.fromFile(s)
    except:
        continue
    for tg_ in tg[0]:
        if tg_.mark is None:
            continue
        if not len(tg_.mark):
            continue
        start.append(tg_.minTime)
        end.append(tg_.maxTime)
        total_l += (tg_.maxTime - tg_.minTime)
        if tg_.mark[0] != '<' and tg_.mark[-1] != '>':
            t = tg_.mark.split()
            filtered = []
            for no, t_ in enumerate(t):
                if t_[-1] == '~':
                    continue
                if t_[0] == '#' and t_[-1] == '#':
                    t_ = t_[1:-1].title()
                filtered.append(t_)
                t[no] = filtered

            texts.append(' '.join(filtered))
        if total_l >= maxlen:
            t = ', '.join(texts)
            if len(t) > 3:
                for r in replaces:
                    t = t.replace(r, '')
                for r, v in replaces_with.items():
                    t = t.replace(r, v)
                t = cleaning(t)
                results.append((t, [start[0], end[-1]]))
            texts = []
            total_l = 0
            start = []
            end = []
    
    if len(texts):
        t = ', '.join(texts)
        if len(t) > 3:
            for r in replaces:
                t = t.replace(r, '')
            for r, v in replaces_with.items():
                t = t.replace(r, v)
            t = cleaning(t)
            results.append((', '.join(texts), [start[0], end[-1]]))
    
    all_results[id_file] = results

100%|██████████| 418/418 [00:20<00:00, 19.93it/s]


In [68]:
keys = list(all_results)
keys = [(k, all_results[k]) for k in keys]

In [69]:
def loop(keys):
    results = []
    keys, _ = keys
    for key in tqdm(keys):
        key, rows = key
        audio_file = os.path.join('part3-separate-audio', f'{key}.wav')
        if not os.path.exists(audio_file):
            continue
        y, sr = sf.read(audio_file)
        index = 0
        for row in rows:
            filename = os.path.join('part3-separate-audio-mp3', f'{key}-{index}.mp3')
            if os.path.exists(filename):
                index += 1
                
                results.append({
                    'filename': filename,
                    'text': row[0]
                })
                
                continue
                
            y_ = y[int(sr * row[-1][0]): int(sr * row[-1][-1])]
            sf.write(filename, y_, sr)
            index += 1
            results.append({
                'filename': filename,
                'text': row[0]
            })
    return results

In [76]:
import mp

results_separate = mp.multiprocessing(keys, loop, cores = 20)

100%|██████████| 20/20 [34:24<00:00, 103.22s/it]


In [78]:
len(results_separate)

169876

In [82]:
results_separate[1000]

{'filename': 'part3-separate-audio-mp3/conf_2501_2501_00862070-106.mp3',
 'text': 'the risk of getting injury is, quite, ya is quite high, ya because of the, roughness, of the ya'}

In [81]:
ipd.Audio(results_separate[1000]['filename'], rate = 16000)

In [83]:
with open('imda-separate-part3.jsonl', 'w') as fopen:
    for r in tqdm(results_separate):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 169876/169876 [00:00<00:00, 240677.84it/s]


In [84]:
from huggingface_hub import HfApi
api = HfApi()

In [85]:
api.upload_file(
    path_or_fileobj='imda-same-part3.jsonl',
    path_in_repo='imda-same-part3.jsonl',
    repo_id='mesolitica/IMDA-STT',
    repo_type='dataset',
)

imda-same-part3.jsonl:   0%|          | 0.00/35.9M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/IMDA-STT/blob/main/imda-same-part3.jsonl'

In [86]:
api.upload_file(
    path_or_fileobj='imda-separate-part3.jsonl',
    path_in_repo='imda-separate-part3.jsonl',
    repo_id='mesolitica/IMDA-STT',
    repo_type='dataset',
)

imda-separate-part3.jsonl:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/IMDA-STT/blob/main/imda-separate-part3.jsonl'

In [87]:
# !sudo apt install p7zip-full p7zip-rar -y

In [88]:
!7z -v5g a part3-same-audio-mp3.7z part3-same-audio-mp3 > /dev/null
!7z -v5g a part3-separate-audio-mp3.7z part3-separate-audio-mp3 > /dev/null

In [89]:
for f in glob('part3-same-audio-mp3.7z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/IMDA-STT',
        repo_type='dataset',
    )

part3-same-audio-mp3.7z.001


part3-same-audio-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part3-same-audio-mp3.7z.002


part3-same-audio-mp3.7z.002:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part3-same-audio-mp3.7z.004


part3-same-audio-mp3.7z.004:   0%|          | 0.00/632M [00:00<?, ?B/s]

part3-same-audio-mp3.7z.003


part3-same-audio-mp3.7z.003:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

In [90]:
for f in glob('part3-separate-audio-mp3.7z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/IMDA-STT',
        repo_type='dataset',
    )

part3-separate-audio-mp3.7z.002


part3-separate-audio-mp3.7z.002:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part3-separate-audio-mp3.7z.003


part3-separate-audio-mp3.7z.003:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

part3-separate-audio-mp3.7z.001


part3-separate-audio-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

In [ ]:
total_len = 0
for f in tqdm(glob('part3-separate-audio/*.wav')):
    y, sr = sf.read(f)
    total_len += len(y) / sr

 13%|█▎        | 58/431 [00:28<02:58,  2.09it/s]

In [ ]:
for f in tqdm(glob('part3-same-audio/*.wav')):
    y, sr = sf.read(f)
    total_len += len(y) / sr

In [ ]:
total_len / 60 / 60

In [ ]:
!rm -rf part3-same-audio part3-separate-audio